<a href="https://colab.research.google.com/github/urvashiramdasani/Document-Summarization/blob/main/notebooks/awesome_text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The aim of this notebook is to provide all possible references to learn Text Summarization. It has been developed after reviewing papers and online material. Feel free to suggest any changes if you think they will improve this notebook!

## Basics of Text Summarization



## Basics of Hidden Markov Models

Hidden Markov Models (HMMs) are a class of probabilistic graphical model that allow us to predict a sequence of unknown (hidden) variables from a set of observed variables. We are constructing an inference model based on the assumptions of a Markov process. The Markov process assumption is simply that the “future is independent of the past given the present”.

Generally, the term “states” are used to refer to the hidden states and “observations” are used to refer to the observed states. The hidden states are also referred to as latent states.

Intution - We compute the joint probability of sequence of hidden states and determine the best possible sequence i.e. the sequence with the highest probability and choose that sequence as the best sequence of hidden states.

1. Transition data — the probability of transitioning to a new state conditioned on a present state.
2. Emission data — the probability of transitioning to an observed state conditioned on a hidden state.
3. Initial state information — the initial probability of transitioning to a hidden state. This can also be looked at as the prior probability.

See Also - Baum Welch Algorithm

In [1]:
# given states - what are the possible combinations
# total number of combinations is (number of possible states)^(sequence length)
def generate_sequence(states,sequence_length):
    
    all_sequences = []
    nodes = []
    
    depth = sequence_length
    
    def gen_seq_recur(states,nodes,depth):
        if depth == 0:
            #print nodes
            all_sequences.append(nodes)
        else:
            for state in states:
                temp_nodes = list(nodes)
                temp_nodes.append(state)
                gen_seq_recur(states,temp_nodes,depth-1)
    
    gen_seq_recur(states,[],depth)
                
    return all_sequences

In [2]:
def score_sequences(sequences,initial_probs,transition_probs,emission_probs,obs):
    
    best_score = -1
    best_sequence = None
    
    sequence_scores = []
    for seq in sequences:
        total_score = 1
        total_score_breakdown = []
        first = True
        for i in range(len(seq)):
            state_score = 1
            # compute transitition probability score
            if first == True:
                state_score *= initial_probs[seq[i]]
                # reset first flag
                first = False
            else:  
                state_score *= transition_probs[seq[i] + "|" + seq[i-1]]
            # add to emission probability score
            state_score *= emission_probs[obs[i] + "|" + seq[i]]
            # update the total score
            #print state_score
            total_score_breakdown.append(state_score)
            total_score *= state_score
            
        sequence_scores.append(total_score)
        
    return sequence_scores

In [5]:
# pretty printing our  distributions
import pandas as pd
from tabulate import tabulate
def pretty_print_probs(distribs):
    
    rows = set()
    cols = set()
    for val in distribs.keys():
        temp = val.split("|")
        rows.add(temp[0])
        cols.add(temp[1])
        
    rows = list(rows)
    cols = list(cols)
    df = []
    for i in range(len(rows)):
        temp = []
        for j in range(len(cols)):
            temp.append(distribs[rows[i]+"|"+cols[j]])
            
        df.append(temp)
        
    I = pd.Index(rows, name="rows")
    C = pd.Index(cols, name="cols")
    df = pd.DataFrame(data=df,index=I, columns=C)
    
    print(tabulate(df, headers='keys', tablefmt='psql'))

In [6]:
def initializeSequences(_obs):
    # Generate list of sequences
    
    seqLen = len(_obs)
    seqs = generate_sequence(states,seqLen)
    # Score sequences
    seq_scores = score_sequences(seqs,initial_probs,transition_probs,emission_probs,obs)
    
    return (seqLen,seqs,seq_scores)

Selecting the best scoring sequence is also known as the Viterbi score. The alternative approach is the Minimum Bayes Risk approach which selects the highest scoring position across all sequence scores.

In [8]:
# We can use a dictionary to capture our state transitions
# set of hidden states
states = ['A','B']
# set of observations
obs = ['Red','Green','Red']
# initial state probability distribution (our priors)
initial_probs = {'A':1.0,'B':0.0}
# transition probabilities
transition_probs = {'A|A':0,'A|B':1,'B|A':1,'B|B':0}
# emission probabilities
emission_probs = {'Red|A':1,'Green|A':0,'Red|B':0.25,'Green|B':0.75}
# Generate list of sequences
sequence_length,sequences,sequence_scores = initializeSequences(obs)
# print results
print("Initial Distributions")
print(initial_probs)
print("\nTransition Probabilities")
pretty_print_probs(transition_probs)
print("\nEmission Probabilities")
pretty_print_probs(emission_probs)
print("\nScores")
# Display sequence scores
for i in range(len(sequences)):
    print("Sequence:%10s,Score:%0.4f" % (sequences[i],sequence_scores[i]))

Initial Distributions
{'A': 1.0, 'B': 0.0}

Transition Probabilities
+--------+-----+-----+
| rows   |   A |   B |
|--------+-----+-----|
| A      |   0 |   1 |
| B      |   1 |   0 |
+--------+-----+-----+

Emission Probabilities
+--------+-----+------+
| rows   |   A |    B |
|--------+-----+------|
| Red    |   1 | 0.25 |
| Green  |   0 | 0.75 |
+--------+-----+------+

Scores
Sequence:['A', 'A', 'A'],Score:0.0000
Sequence:['A', 'A', 'B'],Score:0.0000
Sequence:['A', 'B', 'A'],Score:0.7500
Sequence:['A', 'B', 'B'],Score:0.0000
Sequence:['B', 'A', 'A'],Score:0.0000
Sequence:['B', 'A', 'B'],Score:0.0000
Sequence:['B', 'B', 'A'],Score:0.0000
Sequence:['B', 'B', 'B'],Score:0.0000


POS Tagging

In [9]:
# generate new sequences
states = ['Noun','Verb','Determiner']
initial_probs = {'Noun':0.9,'Verb':0.05,'Determiner':0.05}
transition_probs = {'Noun|Noun':0.1,'Noun|Verb':0.1,'Noun|Determiner':0.8,
                    'Verb|Noun':0.8,'Verb|Verb':0.1,'Verb|Determiner':0.1,
                    'Determiner|Noun':0.1,'Determiner|Verb':0.8,'Determiner|Determiner':0.1}
emission_probs = {'Bob|Noun':0.9,'ate|Noun':0.05,'the|Noun':0.05,'fruit|Noun':0.9,\
                  'Bob|Verb':0.05,'ate|Verb':0.9,'the|Verb':0.05,'fruit|Verb':0.05,\
                  'Bob|Determiner':0.05,'ate|Determiner':0.05,'the|Determiner':0.9,'fruit|Determiner':0.05}
print("Initial Distributions")
print(initial_probs)
print("\nTransition Probabilities")
pretty_print_probs(transition_probs)
print("\nEmission Probabilities")
pretty_print_probs(emission_probs)

Initial Distributions
{'Noun': 0.9, 'Verb': 0.05, 'Determiner': 0.05}

Transition Probabilities
+------------+--------------+--------+--------+
| rows       |   Determiner |   Verb |   Noun |
|------------+--------------+--------+--------|
| Determiner |          0.1 |    0.8 |    0.1 |
| Verb       |          0.1 |    0.1 |    0.8 |
| Noun       |          0.8 |    0.1 |    0.1 |
+------------+--------------+--------+--------+

Emission Probabilities
+--------+--------------+--------+--------+
| rows   |   Determiner |   Verb |   Noun |
|--------+--------------+--------+--------|
| ate    |         0.05 |   0.9  |   0.05 |
| the    |         0.9  |   0.05 |   0.05 |
| Bob    |         0.05 |   0.05 |   0.9  |
| fruit  |         0.05 |   0.05 |   0.9  |
+--------+--------------+--------+--------+


In [10]:
obs = ['Bob','ate','the','fruit']
# print results
print("\nScores")
# Generate list of sequences
sequence_length,sequences,sequence_scores = initializeSequences(obs)
# Display sequence scores
for i in range(len(sequences)):
    print("Sequence:%-60s Score:%0.6f" % (sequences[i],sequence_scores[i]))
    
# Display the winning score
print("\n Best Sequence")
print(sequences[sequence_scores.index(max(sequence_scores))],max(sequence_scores))


Scores
Sequence:['Noun', 'Noun', 'Noun', 'Noun']                             Score:0.000002
Sequence:['Noun', 'Noun', 'Noun', 'Verb']                             Score:0.000001
Sequence:['Noun', 'Noun', 'Noun', 'Determiner']                       Score:0.000000
Sequence:['Noun', 'Noun', 'Verb', 'Noun']                             Score:0.000015
Sequence:['Noun', 'Noun', 'Verb', 'Verb']                             Score:0.000001
Sequence:['Noun', 'Noun', 'Verb', 'Determiner']                       Score:0.000006
Sequence:['Noun', 'Noun', 'Determiner', 'Noun']                       Score:0.000262
Sequence:['Noun', 'Noun', 'Determiner', 'Verb']                       Score:0.000002
Sequence:['Noun', 'Noun', 'Determiner', 'Determiner']                 Score:0.000002
Sequence:['Noun', 'Verb', 'Noun', 'Noun']                             Score:0.000262
Sequence:['Noun', 'Verb', 'Noun', 'Verb']                             Score:0.000117
Sequence:['Noun', 'Verb', 'Noun', 'Determiner']          